### Hackathon 2022 (DDH)
---
- 6개의 Keypoint를 통하여 정상/비정상을 판단하는 문제를 출제한다
- 제공되는 데이터로 6개의 키포인트를 Detection하는 모델을 구축
- 해당 모델에서 찾아진 6개의 키포인트로 비율을 계산해 정상 혹은 비정상을 판단
- 이때의 비율은,
  - 1번 2번 점 이어 연장된 파란색 선
  - 연장된 선과 3번 점의 각도를 A
  - 연장된 선과 4번 점의 각도를 B
  - Head의 최상위, 최하위점 이어서 a가 어딘지 알려 주도록 함
  - 이때 5번 6번 점 이은 직선에서의 a비율을 나눠 정상인지 비정상인지 판단
- Metrics는 oks기반의 mAP를 기준으로 함
- 결과발표를 할 때 어떤 알고리즘을 사용하였으며 어떤 서비스 모델을 만들 수 있을 지 반드시 제시할 것

In [1]:
import json
import os
from pathlib import Path

__file__ = Path(os.path.abspath('__file__'))
root_path = __file__.parent
data_path = root_path / 'data'

sets = ['train', 'validation', 'test']
types = ['images', 'labels']

print(f'{__file__  = }')

__file__  = PosixPath('/home/shane/workspace/hackathon-ddh/__file__')


In [2]:
for s in sets:
    print(s)
    for t in types:
        print(f' {t}\t: ', end='')
        print(len([x for x in (data_path / s / t).glob('**/*') if x.is_file()]))

train
 images	: 8284
 labels	: 8284
validation
 images	: 1000
 labels	: 1000
test
 images	: 1000
 labels	: 1000


In [3]:
for s in sets:
    is_firstline = True
    with open(data_path / f'{s}.csv', 'w') as csv_file:
        for label_path in (x for x in (data_path / s / 'labels').glob('**/*') if x.is_file()):
            keys = []
            values = []
            with open(label_path, 'r') as json_file:
                label = json.load(json_file)
                for k, v in label['images'].items():
                    keys.append(k)
                    values.append(v)
                for k, v in label['annotations'].items():
                    if k == 'detailScore':
                        keys.append(k)
                        values.append(' '.join(map(str, v)))
                    elif k == 'segmentation':
                        keys.append(k)
                        values.append(' '.join(map(str, v[0])))
                    elif k == 'keypoints':
                        keys.append('keypoints')
                        values.append(' '.join(map(str, v['keypoint'])))
                        keys.append('num_keypoints')
                        values.append(v['num_keypoints'])
                        keys.append('containment')
                        values.append(v['containment'])
                        keys.append('diagnose')
                        values.append(v['diagnose'])
                        keys.append('a_angle')
                        values.append(v['a']['angle'])
                        keys.append('a_diagnose')
                        values.append(v['a']['diagnose'])
                        keys.append('b_angle')
                        values.append(v['b']['angle'])
                        keys.append('b_diagnose')
                        values.append(v['b']['diagnose'])
                    else:
                        keys.append(k)
                        values.append(v)

            keys.append('usage')
            values.append(s)

            if is_firstline:
                csv_file.write(','.join(map(str, keys)))
                csv_file.write('\n')
                is_firstline = False
            csv_file.write(','.join(map(str, values)))
            csv_file.write('\n')
